In [1]:
import pandas as pd
import random
#import torch
#from transformers import AutoTokenizer, AutoModel
#import re
#import string
import numpy as np
#import pydot
#from nltk.corpus import stopwords 
#from nltk.stem import WordNetLemmatizer
#from nltk.stem import LancasterStemmer
#from simpletransformers.classification import MultiLabelClassificationModel
#import logging
#import custom_sentence_tokenizer
import matplotlib.pyplot as plt
#from scipy import stats
#from ast import literal_eval
import pickle
from keras.preprocessing.sequence import pad_sequences
import keras.backend as K
import keras
import gc
from keras.models import model_from_json

Using TensorFlow backend.


In [2]:
#from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM, Reshape
from keras.layers import GlobalMaxPooling1D,Conv2D, Conv1D, AveragePooling2D, MaxPooling1D, AveragePooling1D
from keras.models import Model
from keras.utils.vis_utils import plot_model
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.layers import Input
from keras.layers.merge import concatenate
#from sklearn.decomposition import PCA
import _pickle as cPickle
import io
import time
import sys
import boto3
from boto3.s3.transfer import TransferConfig
import tensorflow as tf

## 1. Import Embeddings (X) and Labels (Y)

In [ ]:
now = time.time()
infile = None
infile = open("embeddings.p",'rb')
embeddings = cPickle.load(infile)
infile.close()
del(infile)
print(time.time()-now)
gc.collect()

In [ ]:
embeddings_np_complete = np.array(list(embeddings.items()))
del(embeddings)

## 2. Load chapter model and helper functions

In [ ]:
#Load chapeter model weights
chapter_model = None
json_file = open('model_num_cnn2.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
chapter_model = model_from_json(loaded_model_json)
opt = keras.optimizers.Adam(lr=0.001)
# load weights into new model
chapter_model.load_weights("model_num_cnn2.h5")
chapter_model.trainable = False

In [57]:
def padding(embeddings):
    max_sentence = 20
    padding = max_sentence - len(embeddings)
    if padding > 0:
        padding_shape = (padding, 256, 768)
        pad = np.zeros(padding*256*768).reshape(padding_shape)
        return np.append(embeddings, pad, axis = 0).astype('float16')
    else:
        return embeddings
    
def split_for_train(HADM_ID_LIST, CHAPTER_PIVOT_DF, embeddings_concat):
    HADM_ID_DF = None
    loaded_model = None
    HADM_ID_DF =  pd.DataFrame(data=HADM_ID_LIST, columns=['HADM_ID'])
    CHAPTER_PIVOT_DF_COPY = HADM_ID_DF.set_index('HADM_ID')\
                    .join(CHAPTER_PIVOT_DF, how='left').copy(deep=False)
    CHAPTER_PIVOT_NP = np.array(CHAPTER_PIVOT_DF_COPY)
    return train_test_split(embeddings_concat, CHAPTER_PIVOT_NP, test_size=0.2)

def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

def get_model(label_len):
    #define model - STATIC LAYERS
    model = Sequential()
    for layer in chapter_model.layers[:4]:
        layer.trainable = False
        model.add(layer)

    model = Model(inputs=model.input, outputs=model.output)
    return model

## 3. Loop on individual therapy area (bag of models)

In [ ]:
score_list = []
history_list = []
file_name = 'adverse_effect.csv'

CHAPTER_PIVOT_DF = pd.read_csv('Layer-2/'+file_name, sep=',', header = 0)\
                        .astype({'HADM_ID': 'str'}).set_index('HADM_ID')
HADM_ID_LIST = list(CHAPTER_PIVOT_DF.index)
embeddings_np = embeddings_np_complete[np.isin(embeddings_np_complete[:,0], HADM_ID_LIST)]
print(file_name+' orginal shape: '+str(embeddings_np.shape))
HADM_ID_LIST = embeddings_np[:,0].copy()
embeddings = embeddings_np[:,1].copy()
    
#Zero Pad embeddings
embeddings_padded = [padding(embeddings_ITEM) for embeddings_ITEM in embeddings]
del(embeddings)
gc.collect()
print(file_name+' Padding complete')

#Reshape embeddings
#embeddings_concat = np.array([np.concatenate(i) for i in embeddings_padded])
embeddings_concat = np.array(embeddings_padded).reshape(len(embeddings_padded), 20*256, 768)
print(file_name+' modified shape: '+str(embeddings_concat.shape))
del(embeddings_padded)
gc.collect()
    
X_train2, X_test2, y_train2, y_test2 = split_for_train(HADM_ID_LIST, CHAPTER_PIVOT_DF, embeddings_concat)
print('Train and Test Shapre: '+str(X_train2.shape)+', '+str(X_test2.shape)+', '+str(y_train2.shape)+', '+str(y_test2.shape))
X_val = X_test2[:int(0.25*len(X_test2))]
y_val = y_test2[:int(0.25*len(y_test2))]
del(HADM_ID_LIST)
del(CHAPTER_PIVOT_DF)
del(embeddings_concat)
gc.collect()
    
#call model
label_len = y_train2.shape[1]
model = get_model(label_len)
opt = keras.optimizers.Adam(lr=0.0001)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=[get_f1, 'accuracy'])
flat_X_train = model.predict(X_train2)
#, y_train2, epochs=4,batch_size=32, verbose=1, validation_data = (X_val,y_val))

In [ ]:
 #evaluate model
    print('Model evaluation '+file_name)
    score = model.evaluate(X_test2, y_test2, verbose=1)
    print(score)
    
    #save model evaluation and history metrics
    score_list.append(score)
    history_list.append(history.history)
    
    #save model
    model_json = model.to_json()
    with open("/home/ec2-user/SageMaker/Models/model_"+file_name+".json", "w") as json_file:
        json_file.write(model_json)
    model.save_weights("/home/ec2-user/SageMaker/Models/model_"+file_name+".h5")
    print('============completed '+file_name+'============')